<a href="https://colab.research.google.com/github/daystone/Python/blob/main/data_cleaning_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

df = pd.read_excel('/content/data_file.xlsx', sheet_name = 'Sheet2')
geo = pd.read_excel('/content/data_file.xlsx', sheet_name = "Sheet3")

**Explore Data**

- fill in CLID in df
- make sure dates are in datetime format
- remove C- prefix in geo

In [ ]:
df.dtypes

CLID            object
Date    datetime64[ns]
Vol              int64
dtype: object

In [ ]:
df.describe()

,Date,Vol
count,907,907.000000
mean,2020-10-17 05:01:39.228224768,5376.309813
min,2020-01-31 00:00:00,26.000000
25%,2020-05-31 00:00:00,568.500000
50%,2020-10-31 00:00:00,1173.000000
75%,2021-02-28 00:00:00,5284.500000
max,2021-06-30 00:00:00,41598.000000
std,NaN,8551.586979


In [ ]:
df['Date'].min()

Timestamp('2020-01-31 00:00:00')

In [ ]:
df['Date'].max()

Timestamp('2021-06-30 00:00:00')

In [ ]:
geo.head()

,CLID,GEOID
0,C-CL69323,GEO1001
1,C-CL97995,GEO1001
2,C-CL87299,GEO1003
3,C-CL38496,GEO1001
4,C-CL75562,GEO1003


In [ ]:
geo.dtypes

CLID     object
GEOID    object
dtype: object

In [ ]:
geo.describe()

,CLID,GEOID
count,53,53
unique,53,4
top,C-CL69323,GEO1001
freq,1,20


**Cleaning Data**

In [ ]:
# filling in missing values

df['CLID'].ffill(inplace= True)

# change datatype of date column to datetime format

df['Date'] = pd.to_datetime(df['Date'])

# creating year and quarter columns
df['Year'] = df['Date'].dt.year
df['Quarter'] = df['Date'].dt.quarter
# have to convert year and quarter columns to a string here because you can't concatenate dates
df['Year_Quarter'] = df['Year'].astype(str) + '_' + df['Quarter'].astype(str)

In [ ]:
df.head()

,CLID,Date,Vol,Year,Quarter,Year_Quarter
0,CL11420,2020-03-31,884,2020,1,2020_1
1,CL11420,2020-04-30,886,2020,2,2020_2
2,CL11420,2020-05-31,968,2020,2,2020_2
3,CL11420,2020-06-30,564,2020,2,2020_2
4,CL11420,2020-07-31,648,2020,3,2020_3


**Cleaning Geo Data**

In [ ]:
# removing C- in GEOID column

geo['CLID'] = geo['CLID'].str.replace('C-', '')

In [ ]:
# creating dataframe that pairs CLID with appropirate GEOID

merged_df = df.merge(geo, on = 'CLID')

In [ ]:
merged_df

,CLID,Date,Vol,Year,Quarter,Year_Quarter,GEOID
0,CL11420,2020-03-31,884,2020,1,2020_1,GEO1004
1,CL11420,2020-04-30,886,2020,2,2020_2,GEO1004
2,CL11420,2020-05-31,968,2020,2,2020_2,GEO1004
3,CL11420,2020-06-30,564,2020,2,2020_2,GEO1004
4,CL11420,2020-07-31,648,2020,3,2020_3,GEO1004
...,...,...,...,...,...,...,...
902,CL99768,2021-05-31,290,2021,2,2021_2,GEO1002
903,CL99768,2021-04-30,294,2021,2,2021_2,GEO1002
904,CL99768,2021-03-31,270,2021,1,2021_1,GEO1002
905,CL99768,2021-02-28,224,2021,1,2021_1,GEO1002


**Match GEOIDs with Region Codes**

In [ ]:
# filter data to just see Quarter 2 data for 2021

q2 = merged_df[merged_df['Year_Quarter'] == '2021_2']
q2_vol = q2.groupby('GEOID')['Vol'].sum()
q2_vol

GEOID
GEO1001    596502
GEO1002    109811
GEO1003    176338
GEO1004     82631
Name: Vol, dtype: int64

**Assign Regions to appropriate GEOIDs**

In [ ]:
# create a dictionary

regions = {
    'GEO1001': 'NAM',
    'GEO1002': 'APAC',
    'GEO1003': 'EMEA',
    'GEO1004': 'LATAM'
}

# map region to GEOID

merged_df['Region'] = merged_df['GEOID'].map(regions)
merged_df

,CLID,Date,Vol,Year,Quarter,Year_Quarter,GEOID,Region
0,CL11420,2020-03-31,884,2020,1,2020_1,GEO1004,LATAM
1,CL11420,2020-04-30,886,2020,2,2020_2,GEO1004,LATAM
2,CL11420,2020-05-31,968,2020,2,2020_2,GEO1004,LATAM
3,CL11420,2020-06-30,564,2020,2,2020_2,GEO1004,LATAM
4,CL11420,2020-07-31,648,2020,3,2020_3,GEO1004,LATAM
...,...,...,...,...,...,...,...,...
902,CL99768,2021-05-31,290,2021,2,2021_2,GEO1002,APAC
903,CL99768,2021-04-30,294,2021,2,2021_2,GEO1002,APAC
904,CL99768,2021-03-31,270,2021,1,2021_1,GEO1002,APAC
905,CL99768,2021-02-28,224,2021,1,2021_1,GEO1002,APAC
